In [1]:
import sqlite3
import pandas as pd

In [4]:
conn = sqlite3.connect("bus.db")

In [6]:
df = pd.read_sql("SELECT * FROM sqlite_master", conn)
df

,type,name,tbl_name,rootpage,sql
0,table,boarding,boarding,2,"CREATE TABLE ""boarding"" (\n""index"" INTEGER,\n ..."
1,index,ix_boarding_index,boarding,3,"CREATE INDEX ""ix_boarding_index""ON ""boarding"" ..."
2,table,routes,routes,55,"CREATE TABLE ""routes"" (\n""index"" INTEGER,\n ""..."
3,index,ix_routes_index,routes,57,"CREATE INDEX ""ix_routes_index""ON ""routes"" (""in..."


In [8]:
for cell in df["sql"]:
    print(cell)
    print()

CREATE TABLE "boarding" (
"index" INTEGER,
  "StopID" INTEGER,
  "Route" INTEGER,
  "Lat" REAL,
  "Lon" REAL,
  "DailyBoardings" REAL
)

CREATE INDEX "ix_boarding_index"ON "boarding" ("index")

CREATE TABLE "routes" (
"index" INTEGER,
  "OBJECTID" INTEGER,
  "trips_routes_route_id" INTEGER,
  "route_short_name" INTEGER,
  "route_url" TEXT,
  "ShapeSTLength" REAL
)

CREATE INDEX "ix_routes_index"ON "routes" ("index")



In [9]:
df = pd.read_sql("SELECT * FROM boarding", conn)
df

,index,StopID,Route,Lat,Lon,DailyBoardings
0,0,1163,27,43.073655,-89.385427,1.03
1,1,1163,47,43.073655,-89.385427,0.11
2,2,1163,75,43.073655,-89.385427,0.34
3,3,1164,6,43.106465,-89.340021,10.59
4,4,1167,3,43.077867,-89.369993,3.11
...,...,...,...,...,...,...
3967,3967,6533,67,43.057329,-89.510756,16.88
3968,3968,6539,15,43.064361,-89.517233,15.53
3969,3969,6541,3,43.049934,-89.478167,2.56
3970,3970,6543,70,43.093289,-89.501726,0.11


# Demos

In [25]:
df = pd.read_sql("SELECT DailyBoardings FROM boarding", conn)
df.head()

,DailyBoardings
0,1.03
1,0.11
2,0.34
3,10.59
4,3.11


In [26]:
# SLOW: because it copies all the data from SQL => Python before summming
df.sum()

DailyBoardings    55987.18
dtype: float64

In [27]:
# FAST: calculate in the database, only transfer one number to Python
pd.read_sql("SELECT SUM(DailyBoardings) FROM boarding", conn)

,SUM(DailyBoardings)
0,55987.18


In [34]:
# triple quote: multi line strings
s = """Hello
World"""
print(s)

Hello
World


In [37]:
pd.read_sql("""
SELECT SUM(DailyBoardings)
FROM boarding
WHERE Route = 80
""", conn)

,SUM(DailyBoardings)
0,10211.79


In [41]:
pd.read_sql("""
SELECT *
FROM boarding
ORDER BY lon
LIMIT 1
""", conn)

,index,StopID,Route,Lat,Lon,DailyBoardings
0,3489,4400,55,42.995476,-89.564243,59.31


In [44]:
pd.read_sql("""
SELECT AVG(Lat), AVG(Lon)
FROM boarding
""", conn)

,AVG(Lat),AVG(Lon)
0,43.075374,-89.402466
